In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [7]:
liquidity_factors=get_all_factors(3)

In [8]:
final_res=pd.DataFrame()
r2=pd.Series()
rmse=pd.Series()
alpha={}
tstat={}
for sec in liquidity_factors.columns:
    ind=liquidity_factors.dropna().index
    reges=get_aqr_factors('All').loc[ind]
    reges['Market Return']=cleansed.resample(rule='m',how='last').pct_change().mean(axis=1).dropna()
    reges['const']=1
    #reges['TERM']=quandl.get('FRED/THREEFYTP10',authtoken=token).Value.resample(rule='m',how='last')
    res=sm.OLS(liquidity_factors[sec].loc[ind],reges).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    final_res[sec]=res.params
    final_res[sec+'Tstats']=res.tvalues
    r2[sec]=res.rsquared
    rmse[sec]=math.sqrt(res.mse_model)
    


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  if __name__ == '__main__':


In [9]:
final_res.T.round(4)[['const', u'Market Return','VAL', u'MOM' ]]

,const,Market Return,VAL,MOM
Amihud Monthly,0.0019,0.0426,0.2912,-0.0853
Amihud MonthlyTstats,1.1913,0.5650,2.2499,-0.8242
FHT Monthly,-0.0003,0.2341,-0.0460,-0.1642
FHT MonthlyTstats,-0.1772,2.9385,-0.2836,-1.4496
Amihud Annual,0.0029,-0.0610,0.1743,0.0006
Amihud AnnualTstats,1.6905,-0.7611,1.1339,0.0054
FHT Annual,0.0022,0.1832,-0.4746,-0.3601
FHT AnnualTstats,1.3701,2.4756,-3.2220,-4.0660


In [10]:
print final_res.T.round(2)[['const', u'Market Return','VAL', u'MOM' ]].to_latex()

\begin{tabular}{lrrrr}
\toprule
{} &  const &  Market Return &   VAL &   MOM \\
\midrule
Amihud Monthly       &   0.00 &           0.04 &  0.29 & -0.09 \\
Amihud MonthlyTstats &   1.19 &           0.57 &  2.25 & -0.82 \\
FHT Monthly          &  -0.00 &           0.23 & -0.05 & -0.16 \\
FHT MonthlyTstats    &  -0.18 &           2.94 & -0.28 & -1.45 \\
Amihud Annual        &   0.00 &          -0.06 &  0.17 &  0.00 \\
Amihud AnnualTstats  &   1.69 &          -0.76 &  1.13 &  0.01 \\
FHT Annual           &   0.00 &           0.18 & -0.47 & -0.36 \\
FHT AnnualTstats     &   1.37 &           2.48 & -3.22 & -4.07 \\
\bottomrule
\end{tabular}



In [11]:
(r2*100).round(2)

Amihud Monthly     4.98
FHT Monthly        8.77
Amihud Annual      1.53
FHT Annual        15.56
dtype: float64

In [12]:
rmse.round(3)

Amihud Monthly    0.041
FHT Monthly       0.056
Amihud Annual     0.022
FHT Annual        0.068
dtype: float64